In [3]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.2 MB/s eta 0:00:00


In [7]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

import scipy.sparse as sp

def build_sparse_adj(edge_index, num_nodes):
    edge_index_np = edge_index.numpy()
    adj = sp.coo_matrix(
        (np.ones(edge_index_np.shape[1]), (edge_index_np[0], edge_index_np[1])),
        shape=(num_nodes, num_nodes),
        dtype=np.float32
    )
    # Add self-loops and symmetrize
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    adj = adj + sp.eye(num_nodes)
    # Row-normalize
    rowsum = np.array(adj.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    adj = r_mat_inv.dot(adj)
    # To torch sparse tensor
    adj = adj.tocoo()
    indices = torch.from_numpy(np.vstack((adj.row, adj.col)).astype(np.int64))
    values = torch.from_numpy(adj.data)
    shape = torch.Size(adj.shape)
    return torch.sparse.FloatTensor(indices, values, shape).to(torch.float32)

import numpy as np
adj = build_sparse_adj(data.edge_index, data.num_nodes)

# 4. GCN Layer
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
    def reset_parameters(self):
        nn.init.xavier_uniform_(self.weight)
        if self.bias is not None:
            nn.init.zeros_(self.bias)
    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.sparse.mm(adj, support)
        if self.bias is not None:
            return output + self.bias
        return output

# 5. GCN Model
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super().__init__()
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout
    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

# 6. Training and Evaluation Masks
features = data.x.to(torch.float32)
labels = data.y
idx_train = data.train_mask
idx_val = data.val_mask
idx_test = data.test_mask

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
features = features.to(device)
labels = labels.to(device)
adj = adj.to(device)
idx_train = idx_train.to(device)
idx_val = idx_val.to(device)
idx_test = idx_test.to(device)

model = GCN(nfeat=features.shape[1], nhid=16, nclass=dataset.num_classes, dropout=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def accuracy(output, labels):
    preds = output.max(1)[1]
    return preds.eq(labels).sum().item() / len(labels)

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    model.eval()
    output = model(features, adj)
    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, loss_train: {loss_train.item():.4f}, loss_val: {loss_val.item():.4f}, val_acc: {acc_val:.4f}')

# Final test
model.eval()
output = model(features, adj)
loss_test = F.nll_loss(output[idx_test], labels[idx_test])
acc_test = accuracy(output[idx_test], labels[idx_test])
print(f"Test set results: loss = {loss_test.item():.4f}, accuracy = {acc_test:.4f}")

Epoch: 000, loss_train: 1.9411, loss_val: 1.8884, val_acc: 0.4480
Epoch: 010, loss_train: 0.6930, loss_val: 1.0563, val_acc: 0.7560
Epoch: 020, loss_train: 0.2268, loss_val: 0.7289, val_acc: 0.7880
Epoch: 030, loss_train: 0.0675, loss_val: 0.7161, val_acc: 0.7860
Epoch: 040, loss_train: 0.0485, loss_val: 0.7392, val_acc: 0.7800
Epoch: 050, loss_train: 0.0445, loss_val: 0.7428, val_acc: 0.7820
Epoch: 060, loss_train: 0.0217, loss_val: 0.7554, val_acc: 0.7740
Epoch: 070, loss_train: 0.0408, loss_val: 0.7736, val_acc: 0.7680
Epoch: 080, loss_train: 0.0490, loss_val: 0.7758, val_acc: 0.7680
Epoch: 090, loss_train: 0.0280, loss_val: 0.7600, val_acc: 0.7680
Epoch: 100, loss_train: 0.0245, loss_val: 0.7727, val_acc: 0.7720
Epoch: 110, loss_train: 0.0274, loss_val: 0.7767, val_acc: 0.7680
Epoch: 120, loss_train: 0.0426, loss_val: 0.7783, val_acc: 0.7640
Epoch: 130, loss_train: 0.0307, loss_val: 0.7630, val_acc: 0.7740
Epoch: 140, loss_train: 0.0293, loss_val: 0.7864, val_acc: 0.7640
Epoch: 150